In [1]:
#Calculate energy and forces with a machine learning
#potential, for a given structure

In [1]:
import os
import shutil
import numpy as np
from sklearn import preprocessing
from sklearn.externals import joblib
import mlmd.tools.readers as readers
import mlmd.tools.builders as builders
import mlmd.machine_learning.mlt as mlt
import mlmd.machine_learning.mlp as mlp
import mlmd.MD_suit.MD_suit as MD

In [2]:
#loads feature parameters from dire_expe_name
dire_expe_name='training_data_test'
inpu_stru_path= 'input.stru'
feature_parameters= np.load(dire_expe_name+'/feature_parameters.npy').item()
trans = feature_parameters['trans']
eta2b = feature_parameters['eta2b']
Rp = feature_parameters['Rp']
eta3b = feature_parameters['eta3b']
cos_p = feature_parameters['cos_p']
stru_name= []
#loads initial structure
stru_symb, stru_posi= readers.load_md_init_stru(inpu_stru_path)

#calculates feature representation for a given initial structure
feat_2b, feat_3b,X, DX= builders.build_SIFF_DSIFF(trans, eta2b, Rp, eta3b, cos_p,\
                                                  stru_symb, stru_name, stru_posi)

/home/accounts/zeledon/anaconda2/lib/python2.7/site-packages/mlmd-0.1-py2.7-linux-x86_64.egg/mlmd/tools/geometry.py:15: RuntimeWarning: invalid value encountered in divide
/home/accounts/zeledon/anaconda2/lib/python2.7/site-packages/mlmd-0.1-py2.7-linux-x86_64.egg/mlmd/tools/geometry.py:16: RuntimeWarning: divide by zero encountered in divide
/home/accounts/zeledon/anaconda2/lib/python2.7/site-packages/mlmd-0.1-py2.7-linux-x86_64.egg/mlmd/tools/geometry.py:18: RuntimeWarning: divide by zero encountered in divide
/home/accounts/zeledon/anaconda2/lib/python2.7/site-packages/mlmd-0.1-py2.7-linux-x86_64.egg/mlmd/tools/geometry.py:94: RuntimeWarning: invalid value encountered in multiply
/home/accounts/zeledon/anaconda2/lib/python2.7/site-packages/mlmd-0.1-py2.7-linux-x86_64.egg/mlmd/tools/geometry.py:97: RuntimeWarning: invalid value encountered in multiply


In [3]:
print trans

{'C': 6, 'O': 8}


In [4]:
print stru_symb

[['C', 'C', 'C', 'C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [5]:
#create potential object
potential= mlp.mlp(dire_expe_name)
#loads E and F potentials (GBR models)
potential.load_GBR_E_potential(dire_expe_name)
potential.load_GBR_F_potential(dire_expe_name)

#scale the feature representation
#with the trainned scalers
X_scaled= potential.scaler_E.transform(X)
DXx_scaled= potential.scaler_Fx.transform(DX[0,:,:,0])
DXy_scaled= potential.scaler_Fy.transform(DX[0,:,:,1])
DXz_scaled= potential.scaler_Fz.transform(DX[0,:,:,2])

In [6]:
#predict E and F with ML potential for the inital structure
E_pred= potential.predict_E_GBR(X_scaled)
F_pred= potential.predict_F_GBR(DXx_scaled, DXy_scaled, DXz_scaled)

In [7]:
# molecular dynamics part

#Load md parameters
Qmass, temp, dt, correc_steps, md_steps, exp_name= readers.get_md_parameters('input.md')

#initialization of MD variables
amu= readers.get_amu(stru_symb, trans)
nat= len(stru_symb[0])
s_in=1.0 #thermostat degree of freedom
s_in_dot= 0.0#time derivative of thermostat degree of freedom
r_in= stru_posi[0].T
fcart_in= F_pred.T
v_in= MD.md_suit.init_vel_atoms(amu, temp,nat) #check units

In [8]:
#MD loop
path_to_xyz='md_path.xyz'
s_arra= []
for _ in range(md_steps):
    s_out, s_out_dot, r_out, v_out= MD.md_suit.md_nvt(r_in,fcart_in, v_in, \
                                                      amu, Qmass, dt, temp, s_in, s_in_dot,\
                                                      correc_steps, 1,nat)
    #print get_V_cm(amu, v_in)
    #print '*********'
    #recalculate forces and energies
    #calculate features for r_out
    #print r_out.T
    #print '*********'
    stru_t= np.array([r_out.T])
    feat_2b, feat_3b,X, DX= builders.build_SIFF_DSIFF(trans, eta2b, Rp, eta3b, cos_p,\
                                                      stru_symb, stru_name, stru_t)
    #scale features from r_out
    X_scaled= potential.scaler_E.transform(X)
    DXx_scaled= potential.scaler_Fx.transform(DX[0,:,:,0])
    DXy_scaled= potential.scaler_Fy.transform(DX[0,:,:,1])
    DXz_scaled= potential.scaler_Fz.transform(DX[0,:,:,2])
    #predict E and F with ML potential for r_out
    E_pred= potential.predict_E_GBR(X_scaled)
    F_pred= potential.predict_F_GBR(DXx_scaled, DXy_scaled, DXz_scaled)

    temp= readers.get_temp(amu, v_out)
    readers.write_xyz(path_to_xyz,  E_pred, temp, stru_symb, r_in)

    fcart_in= F_pred.T
    s_in= s_out
    s_arra.append(s_out)
    s_in_dot= s_out_dot
    r_in= r_out
    v_in= v_out